# Optimal Temperature Scaling for Noisy Language Models

**Theory:** For a noisy model with logits z̃ = z + ε, the optimal temperature is:

$$T^* = \sqrt{1 + \alpha}$$

where α = σ²_noise / τ²_signal is the noise-to-signal ratio.

**Experiments:**
1. Weight noise only
2. Activation noise only
3. Compare weight vs activation
4. Combined noise (additivity test)

In [ ]:
# ============================================================
# UTILITIES - Run this cell first!
# ============================================================

"""
Utility functions for quantization noise and temperature experiments.
"""

import torch
import torch.nn.functional as F
import numpy as np
from contextlib import contextmanager
import re


# =============================================================================
# Noise Injection
# =============================================================================

@contextmanager
def weight_noise_context(model, noise_scale):
    """
    Context manager that adds Gaussian noise to model weights.
    Noise is removed when exiting the context.
    """
    original_weights = {}
    
    try:
        # Add noise to weights
        for name, param in model.named_parameters():
            if 'weight' in name and param.requires_grad is False:
                original_weights[name] = param.data.clone()
                noise = torch.randn_like(param.data) * noise_scale * param.data.std()
                param.data.add_(noise)
        
        yield model, original_weights
        
    finally:
        # Restore original weights
        for name, param in model.named_parameters():
            if name in original_weights:
                param.data.copy_(original_weights[name])


@contextmanager  
def activation_noise_context(model, noise_scale):
    """
    Context manager that adds Gaussian noise to activations via forward pre-hooks.
    Uses INPUT hooks (pre-hook) for realistic quantization simulation.
    """
    handles = []
    
    def make_hook(scale):
        def hook(module, args, kwargs):
            if len(args) > 0:
                x = args[0]
                noise = torch.randn_like(x) * scale * x.std()
                noisy_x = x + noise
                return (noisy_x,) + args[1:], kwargs
            return args, kwargs
        return hook
    
    try:
        # Register pre-hooks on linear layers
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                handle = module.register_forward_pre_hook(make_hook(noise_scale), with_kwargs=True)
                handles.append(handle)
        
        yield model, handles
        
    finally:
        # Remove hooks
        for handle in handles:
            handle.remove()


@contextmanager
def combined_noise_context(model, weight_noise_scale, activation_noise_scale):
    """
    Context manager that adds both weight and activation noise.
    """
    handles = []
    original_weights = {}
    
    def make_hook(scale):
        def hook(module, args, kwargs):
            if len(args) > 0:
                x = args[0]
                noise = torch.randn_like(x) * scale * x.std()
                noisy_x = x + noise
                return (noisy_x,) + args[1:], kwargs
            return args, kwargs
        return hook
    
    try:
        # Add weight noise
        for name, param in model.named_parameters():
            if 'weight' in name and param.requires_grad is False:
                original_weights[name] = param.data.clone()
                noise = torch.randn_like(param.data) * weight_noise_scale * param.data.std()
                param.data.add_(noise)
        
        # Add activation hooks
        for name, module in model.named_modules():
            if isinstance(module, torch.nn.Linear):
                handle = module.register_forward_pre_hook(make_hook(activation_noise_scale), with_kwargs=True)
                handles.append(handle)
        
        yield model, (original_weights, handles)
        
    finally:
        # Restore weights
        for name, param in model.named_parameters():
            if name in original_weights:
                param.data.copy_(original_weights[name])
        
        # Remove hooks
        for handle in handles:
            handle.remove()


# =============================================================================
# Alpha (noise-to-signal ratio) Measurement
# =============================================================================

def compute_alpha(clean_logits, noisy_logits):
    """
    Compute noise-to-signal ratio α = σ²_noise / τ²_signal
    
    Args:
        clean_logits: Logits from clean model
        noisy_logits: Logits from noisy model
    
    Returns:
        dict with tau_sq, sigma_sq, alpha, t_star
    """
    diff = noisy_logits - clean_logits
    tau_sq = clean_logits.var().item()
    sigma_sq = diff.var().item()
    alpha = sigma_sq / tau_sq if tau_sq > 0 else 0
    t_star = np.sqrt(1 + alpha)
    
    return {
        'tau_sq': tau_sq,
        'sigma_sq': sigma_sq,
        'alpha': alpha,
        't_star': t_star
    }


def get_logits(model, tokenizer, prompt):
    """Get logits for next token prediction."""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model(inputs.input_ids)
    return outputs.logits[0, -1, :].float()


# =============================================================================
# Generation with Temperature (FIXED - uses sampling!)
# =============================================================================

def generate_with_temperature(model, tokenizer, prompt, temperature=1.0, 
                              max_new_tokens=512, do_sample=True, seed=None):
    """
    Generate text with proper temperature scaling.
    
    IMPORTANT: Temperature only affects sampling, not argmax!
    - do_sample=True: temperature scales probabilities before sampling
    - do_sample=False: temperature has NO effect (argmax is scale-invariant)
    
    Args:
        model: The language model
        tokenizer: The tokenizer
        prompt: Input prompt string
        temperature: Sampling temperature (only works with do_sample=True)
        max_new_tokens: Maximum tokens to generate
        do_sample: If True, sample from distribution. If False, greedy (T ignored!)
        seed: Random seed for reproducibility
    
    Returns:
        generated_text: The generated string
        generated_tokens: List of token ids
    """
    if seed is not None:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
    
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_ids = inputs.input_ids.clone()
    generated_tokens = []
    
    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(input_ids)
            logits = outputs.logits[0, -1, :].float()
            
            if do_sample and temperature > 0:
                # Apply temperature and sample
                scaled_logits = logits / temperature
                probs = F.softmax(scaled_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1).item()
            else:
                # Greedy - temperature has NO effect!
                next_token = logits.argmax().item()
            
            generated_tokens.append(next_token)
            input_ids = torch.cat([input_ids, torch.tensor([[next_token]]).to(model.device)], dim=1)
            
            if next_token == tokenizer.eos_token_id:
                break
    
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text, generated_tokens


def generate_greedy(model, tokenizer, prompt, max_new_tokens=512):
    """
    Generate text with greedy decoding (deterministic).
    Temperature has NO effect on greedy decoding!
    """
    return generate_with_temperature(
        model, tokenizer, prompt, 
        temperature=1.0, max_new_tokens=max_new_tokens, 
        do_sample=False
    )


# =============================================================================
# Evaluation Metrics
# =============================================================================

def evaluate_temperature_effect(clean_logits, noisy_logits, temperatures):
    """
    Evaluate how different temperatures affect the noisy distribution.
    
    For each temperature, computes:
    - KL divergence from clean distribution
    - JS divergence (symmetric)
    - Probability of correct (clean's argmax) token
    - Cross entropy
    
    Args:
        clean_logits: Logits from clean model
        noisy_logits: Logits from noisy model  
        temperatures: List of temperatures to test
    
    Returns:
        dict mapping temperature to metrics
    """
    clean_probs = F.softmax(clean_logits, dim=-1)
    correct_token = clean_logits.argmax().item()
    
    results = {}
    
    for temp in temperatures:
        noisy_probs = F.softmax(noisy_logits / temp, dim=-1)
        
        # KL divergence: KL(clean || noisy)
        kl_div = F.kl_div(noisy_probs.log(), clean_probs, reduction='sum').item()
        
        # JS divergence (symmetric)
        m_probs = 0.5 * (clean_probs + noisy_probs)
        js_div = 0.5 * F.kl_div(m_probs.log(), clean_probs, reduction='sum').item() + \
                 0.5 * F.kl_div(m_probs.log(), noisy_probs, reduction='sum').item()
        
        # Probability of correct token
        prob_correct = noisy_probs[correct_token].item()
        
        # Cross entropy
        cross_entropy = -(clean_probs * noisy_probs.log()).sum().item()
        
        results[temp] = {
            'kl_div': kl_div,
            'js_div': js_div,
            'prob_correct': prob_correct,
            'cross_entropy': cross_entropy,
        }
    
    return results


# =============================================================================
# GSM8K Helpers
# =============================================================================

def extract_answer(text):
    """Extract numerical answer from model output."""
    # Look for #### pattern (GSM8K format)
    match = re.search(r'####\s*([\d,\.\-]+)', text)
    if match:
        return match.group(1).replace(',', '')
    
    # Look for 'answer is X' pattern
    match = re.search(r'answer is[:\s]*([\d,\.\-]+)', text.lower())
    if match:
        return match.group(1).replace(',', '')
    
    # Look for boxed answer (common in reasoning models)
    match = re.search(r'\\boxed\{([\d,\.\-]+)\}', text)
    if match:
        return match.group(1).replace(',', '')
    
    # Look for last number in text
    numbers = re.findall(r'[\d,]+\.?\d*', text)
    if numbers:
        return numbers[-1].replace(',', '')
    
    return None


def extract_ground_truth(answer_text):
    """Extract ground truth from GSM8K answer format."""
    match = re.search(r'####\s*([\d,\.\-]+)', answer_text)
    if match:
        return match.group(1).replace(',', '')
    return None


def check_answer(pred, truth):
    """Check if predicted answer matches ground truth."""
    if pred is None or truth is None:
        return False
    try:
        return abs(float(pred) - float(truth)) < 0.01
    except:
        return pred.strip() == truth.strip()


def format_gsm8k_prompt(question, tokenizer, use_chat_template=True):
    """Format GSM8K question as prompt."""
    if use_chat_template and hasattr(tokenizer, 'apply_chat_template'):
        messages = [
            {"role": "user", "content": f"Solve this math problem step by step:\n{question}"}
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        return f"Question: {question}\nLet me solve this step by step:\n"


# =============================================================================
# Reproducibility
# =============================================================================

def set_seed(seed):
    """Set random seed for reproducibility."""
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# =============================================================================
# Sanity Checks
# =============================================================================

def verify_noise_injection(model, tokenizer, noise_ctx_fn, noise_scale, prompt="Hello"):
    """
    Verify that noise injection is actually working.
    
    Returns True if noise is being applied (outputs differ).
    """
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    
    # Clean output
    with torch.no_grad():
        clean_out = model(inputs.input_ids).logits[0, -1, :]
    
    # Noisy outputs (should differ each time)
    noisy_outs = []
    for _ in range(3):
        with noise_ctx_fn(model, noise_scale) as (noisy_model, _):
            with torch.no_grad():
                noisy_out = noisy_model(inputs.input_ids).logits[0, -1, :]
                noisy_outs.append(noisy_out.clone())
    
    # Check that noisy differs from clean
    diff_from_clean = (noisy_outs[0] - clean_out).abs().mean().item()
    
    # Check that noisy outputs differ from each other (stochastic)
    diff_between_noisy = (noisy_outs[0] - noisy_outs[1]).abs().mean().item()
    
    print(f"Mean diff from clean: {diff_from_clean:.4f}")
    print(f"Mean diff between noisy runs: {diff_between_noisy:.4f}")
    
    is_working = diff_from_clean > 0.01 and diff_between_noisy > 0.01
    print(f"Noise injection working: {is_working}")
    
    return is_working


In [ ]:
# Configuration
MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"  # Change as needed
SEED = 42
set_seed(SEED)

print(f"Loading {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map="auto"
)
model.eval()
print(f"Model loaded on {model.device}")

In [ ]:
# Verify noise injection
print("Sanity check - verifying noise injection...")
print("\nWeight noise:")
verify_noise_injection(model, tokenizer, weight_noise_context, 0.05)
print("\nActivation noise:")
verify_noise_injection(model, tokenizer, activation_noise_context, 0.05)

In [ ]:
# Test prompts
TEST_PROMPTS = [
    "The capital of France is",
    "2 + 2 =",
    "The largest planet in our solar system is",
    "Water freezes at",
]

# Noise scales to test
NOISE_SCALES = [0.01, 0.02, 0.05, 0.1, 0.2]

In [ ]:
# Get clean baseline logits
print("Getting clean baseline logits...")
clean_logits = {}
for prompt in TEST_PROMPTS:
    clean_logits[prompt] = get_logits(model, tokenizer, prompt)
print(f"Got logits for {len(TEST_PROMPTS)} prompts")

## Experiment 1: Weight Noise Only

Add Gaussian noise to model weights and measure α.

In [ ]:
print("Experiment 1: Weight Noise")
print("="*60)

weight_results = []

for noise_scale in NOISE_SCALES:
    alphas = []
    
    for prompt in TEST_PROMPTS:
        with weight_noise_context(model, noise_scale) as (noisy_model, _):
            noisy_logits = get_logits(noisy_model, tokenizer, prompt)
        
        result = compute_alpha(clean_logits[prompt], noisy_logits)
        alphas.append(result['alpha'])
    
    mean_alpha = np.mean(alphas)
    t_star = np.sqrt(1 + mean_alpha)
    
    weight_results.append({
        'noise_scale': noise_scale,
        'alpha': mean_alpha,
        't_star': t_star,
    })
    
    print(f"σ={noise_scale:.3f}: α={mean_alpha:.4f}, T*={t_star:.4f}")

## Experiment 2: Activation Noise Only

Add Gaussian noise to activations (at layer inputs) and measure α.

In [ ]:
print("Experiment 2: Activation Noise")
print("="*60)

activation_results = []

for noise_scale in NOISE_SCALES:
    alphas = []
    
    for prompt in TEST_PROMPTS:
        with activation_noise_context(model, noise_scale) as (noisy_model, _):
            noisy_logits = get_logits(noisy_model, tokenizer, prompt)
        
        result = compute_alpha(clean_logits[prompt], noisy_logits)
        alphas.append(result['alpha'])
    
    mean_alpha = np.mean(alphas)
    t_star = np.sqrt(1 + mean_alpha)
    
    activation_results.append({
        'noise_scale': noise_scale,
        'alpha': mean_alpha,
        't_star': t_star,
    })
    
    print(f"σ={noise_scale:.3f}: α={mean_alpha:.4f}, T*={t_star:.4f}")

## Experiment 3: Compare Weight vs Activation Noise

In [ ]:
print("Experiment 3: Comparison")
print("="*60)
print(f"{'Noise σ':<10} {'Weight α':<12} {'Activ α':<12} {'Ratio':<10} {'Weight T*':<10} {'Activ T*':<10}")
print("-"*60)

for w, a in zip(weight_results, activation_results):
    ratio = a['alpha'] / w['alpha'] if w['alpha'] > 0 else float('inf')
    print(f"{w['noise_scale']:<10.3f} {w['alpha']:<12.4f} {a['alpha']:<12.4f} {ratio:<10.1f}x {w['t_star']:<10.4f} {a['t_star']:<10.4f}")

avg_ratio = np.mean([a['alpha']/w['alpha'] for w, a in zip(weight_results, activation_results) if w['alpha'] > 0])
print(f"\nAverage: Activation noise produces {avg_ratio:.1f}x larger α than weight noise!")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

noise_scales = [r['noise_scale'] for r in weight_results]
weight_alphas = [r['alpha'] for r in weight_results]
activ_alphas = [r['alpha'] for r in activation_results]
weight_tstars = [r['t_star'] for r in weight_results]
activ_tstars = [r['t_star'] for r in activation_results]

# Left: α vs noise scale
ax1 = axes[0]
ax1.plot(noise_scales, weight_alphas, 'b-o', label='Weight noise', linewidth=2, markersize=8)
ax1.plot(noise_scales, activ_alphas, 'r-s', label='Activation noise', linewidth=2, markersize=8)
ax1.set_xlabel('Noise Scale σ')
ax1.set_ylabel('α (noise-to-signal ratio)')
ax1.set_title('α vs Noise Scale')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Right: T* vs noise scale
ax2 = axes[1]
ax2.plot(noise_scales, weight_tstars, 'b-o', label='Weight noise', linewidth=2, markersize=8)
ax2.plot(noise_scales, activ_tstars, 'r-s', label='Activation noise', linewidth=2, markersize=8)
ax2.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='T=1')
ax2.set_xlabel('Noise Scale σ')
ax2.set_ylabel('T* = √(1+α)')
ax2.set_title('Optimal Temperature vs Noise Scale')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('weight_vs_activation_noise.png', dpi=150, bbox_inches='tight')
plt.show()

## Experiment 4: Combined Noise (Additivity Test)

Test if α_combined ≈ α_weight + α_activation

In [ ]:
print("Experiment 4: Noise Additivity")
print("="*60)

test_noise = 0.05  # Use middle noise level

# Get individual α values
w_result = next(r for r in weight_results if r['noise_scale'] == test_noise)
a_result = next(r for r in activation_results if r['noise_scale'] == test_noise)

# Measure combined α
combined_alphas = []
for prompt in TEST_PROMPTS:
    with combined_noise_context(model, test_noise, test_noise) as (noisy_model, _):
        noisy_logits = get_logits(noisy_model, tokenizer, prompt)
    
    result = compute_alpha(clean_logits[prompt], noisy_logits)
    combined_alphas.append(result['alpha'])

combined_alpha = np.mean(combined_alphas)
expected_sum = w_result['alpha'] + a_result['alpha']

print(f"Weight α:      {w_result['alpha']:.4f}")
print(f"Activation α:  {a_result['alpha']:.4f}")
print(f"Expected sum:  {expected_sum:.4f}")
print(f"Combined α:    {combined_alpha:.4f}")
print(f"Ratio:         {combined_alpha/expected_sum:.2f}x")

if 0.8 < combined_alpha/expected_sum < 1.2:
    print("\n✓ Noise sources are approximately ADDITIVE!")
else:
    print("\n✗ Noise sources are NOT additive (interaction effects present)")

In [ ]:
# Bar chart for additivity
fig, ax = plt.subplots(figsize=(8, 6))

categories = ['Weight\nonly', 'Activation\nonly', 'Expected\nsum', 'Combined\n(actual)']
values = [w_result['alpha'], a_result['alpha'], expected_sum, combined_alpha]
colors = ['blue', 'red', 'gray', 'purple']

bars = ax.bar(categories, values, color=colors, alpha=0.7)
ax.set_ylabel('α (noise-to-signal ratio)')
ax.set_title(f'Noise Additivity Test (σ={test_noise})')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{val:.3f}', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.savefig('noise_additivity.png', dpi=150, bbox_inches='tight')
plt.show()

## Summary Table

In [ ]:
print("\n" + "="*70)
print("SUMMARY: Weight vs Activation Noise")
print("="*70)
print(f"\n{'Noise σ':<10} {'Type':<12} {'α':<12} {'T*':<10}")
print("-"*44)

for w, a in zip(weight_results, activation_results):
    print(f"{w['noise_scale']:<10.3f} {'Weight':<12} {w['alpha']:<12.4f} {w['t_star']:<10.3f}")
    print(f"{'':<10} {'Activation':<12} {a['alpha']:<12.4f} {a['t_star']:<10.3f}")

print("\n" + "="*70)
print("KEY FINDINGS:")
print("="*70)
print(f"1. Activation noise produces ~{avg_ratio:.0f}x larger α than weight noise")
print(f"2. For σ=0.05: Weight T*={weight_results[2]['t_star']:.3f}, Activation T*={activation_results[2]['t_star']:.3f}")
print(f"3. Noise sources are {'additive' if 0.8 < combined_alpha/expected_sum < 1.2 else 'NOT additive'}")
print("="*70)

## Next Steps

See `experiment_5_temperature.ipynb` for:
- Temperature validation with **sampling** (not greedy!)
- GSM8K accuracy evaluation
- Position-dependent temperature